In [1]:
import requests

from bs4 import BeautifulSoup

import bs4

import pandas as pd

import numpy as np

from IPython.display import Image

In [3]:
url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1760&minBedrooms=4&radius=1.0&propertyTypes=detached%2Csemi-detached%2Cterraced&primaryDisplayPropertyType=houses&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='
r = requests.get(url)
r.text[:300]

In [9]:
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify()[:1000])

### step0
get bedrooms, addresses and price per month, individually

In [14]:
house_type =[]
for num in soup.find_all('h2', class_= 'propertyCard-title'):
    house_type.append(num.text.strip())
#     print(num.text)

In [22]:
print(len(house_type))
print(house_type[:5])

25
['5 bedroom house', '5 bedroom house share', '5 bedroom house share', '4 bedroom semi-detached house', '8 bedroom house share']


In [16]:
house_address = []
for add in soup.find_all('address', class_="propertyCard-address"):
    for meta in add:
        for loc in meta:
            if type(loc) is bs4.element.NavigableString:
                house_address.append(loc)

In [20]:
print(len(house_address))
print(house_address[:5])

25
['St Michaels Road, Canterbury', 'GREEN DELL', 'DOWNS ROAD', 'Tenterden Drive, Canterbury, Kent, CT2', 'Saint Dunstans Street']


In [23]:
pcm = []
for pc in soup.find_all('span', 'propertyCard-priceValue'):
    pcm.append(pc.string)
print(len(pcm))
print(pcm[:5])

25
['£1,780 pcm', '£390 pcm', '£425 pcm', '£420 pcm', '£365 pcm']


In [33]:
# test the page numbers
for start in range(24,72,24):
    print(start)

24
48


### step1
worked

In [28]:
for start in range(24,72,24):
    url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1760&minBedrooms=4&radius=1.0&index={}&propertyTypes=detached%2Csemi-detached%2Cterraced&primaryDisplayPropertyType=houses&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='.format(start)
    r = requests.get(url)
#     print(r.text)

### step2
worked

In [29]:
bdrooms = []

for start in range(24,72,24):
    url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1760&minBedrooms=4&radius=1.0&index={}&propertyTypes=detached%2Csemi-detached%2Cterraced&primaryDisplayPropertyType=houses&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='.format(start)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    for num in soup.find_all('h2', class_= 'propertyCard-title'):
        bdrooms.append(num.text.strip())
    

In [32]:
print(len(bdrooms))
bdrooms[:5]

50


['4 bedroom semi-detached house',
 '4 bedroom house',
 '6 bedroom house share',
 '5 bedroom house share',
 '4 bedroom house share']

### step3
worked

In [47]:
bdrooms = []
addresses = []
rent = []

for start in range(0,480,24):
    url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1760&minBedrooms=4&radius=1.0&index={}&propertyTypes=detached%2Csemi-detached%2Cterraced&primaryDisplayPropertyType=houses&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='.format(start)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    for num in soup.find_all('h2', class_= 'propertyCard-title'):
        bdrooms.append(num.text.strip())
        
    for add in soup.find_all('address', class_="propertyCard-address"):
        for meta in add:
            for loc in meta:
                if type(loc) is bs4.element.NavigableString:
                    addresses.append(loc)
                    
    for pc in soup.find_all('span', 'propertyCard-priceValue'):
        rent.append(pc.string)

TypeError: 'NoneType' object is not iterable

In [49]:
print(len(bdrooms))
print(bdrooms[:5])
print(len(addresses))
print(addresses[:5])
print(len(rent))
print(rent[:5])

500
['4 bedroom semi-detached house', '5 bedroom semi-detached house', '5 bedroom house share', '5 bedroom house share', '4 bedroom semi-detached house']
495
['Mead Way, Canterbury', 'Westerham Close, Canterbury, CT2', 'GREEN DELL', 'DOWNS ROAD', 'Tenterden Drive, Canterbury, Kent, CT2']
500
['£1,640 pcm', '£1,870 pcm', '£390 pcm', '£425 pcm', '£420 pcm']


### step4

In [51]:
canterbury_west_station_1miles = pd.DataFrame({'bdrooms':pd.Series(bdrooms), 'address':pd.Series(addresses), 'pcm':pd.Series(rent)})
canterbury_west_station_1miles.head()
canterbury_west_station_1miles.tail()

,bdrooms,address,pcm
495,4 bedroom house,NaN,"£1,596 pcm"
496,4 bedroom house,NaN,"£1,596 pcm"
497,4 bedroom house,NaN,"£1,620 pcm"
498,5 bedroom house,NaN,"£1,995 pcm"
499,Property,NaN,None


In [57]:
canterbury_west_station_1miles = canterbury_west_station_1miles.dropna()

In [59]:
canterbury_west_station_1miles['pcm'] = [int(''.join(filter(str.isdigit, i))) for i in canterbury_west_station_1miles['pcm']]

<ipython-input-59-66fda437f5fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canterbury_west_station_1miles['pcm'] = [int(''.join(filter(str.isdigit, i))) for i in canterbury_west_station_1miles['pcm']]


In [60]:
canterbury_west_station_1miles

,bdrooms,address,pcm
0,4 bedroom semi-detached house,"Mead Way, Canterbury",1640
1,5 bedroom semi-detached house,"Westerham Close, Canterbury, CT2",1870
2,5 bedroom house share,GREEN DELL,390
3,5 bedroom house share,DOWNS ROAD,425
4,4 bedroom semi-detached house,"Tenterden Drive, Canterbury, Kent, CT2",420
...,...,...,...
490,5 bedroom terraced house,"Payton Mews, Canterbury, Kent",2300
491,5 bedroom house,"Mary Green Walk, Canterbury, Kent",2025
492,4 bedroom terraced house,"Jessica Mews, Canterbury, Kent",1490
493,4 bedroom house,"Hudson Road, Canterbury, Kent",1700


In [61]:
canterbury_west_station_1miles.describe()

,pcm
count,491.000000
mean,1558.991853
std,719.683666
min,300.000000
25%,1330.000000
50%,1612.000000
75%,2000.000000
max,4465.000000
